<a href="https://colab.research.google.com/github/silverstar0727/study-/blob/master/Auto_Grad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AUTOGRAD(자동미분)
pytorch의 모든 신경망의 중심에는 autograd패키지가 존재한다. 이는 tensor의 모든 연산에 대해 자동 미분을 제공한다. 실행에 기반한 정의(define-by-run) 프레임워크로 코드를 어떻게 작성하는지에 따라 역전파가 정의된다는 것을 의미하고, 역전파는 학습 과정의 단계마다 달라진다.

## Tensor
torch.Tensor 클래스에서 .requires_grad 속성을 True로 설정하면 그 tensor에서 이뤄진 모든 연산을 추적(track)하기 시작한다. 계산이 끝나면 .backward()를 호출하여 gradient를 자동으로 계산할 수 있으며, .grad속성에 누적된다.

이를 중단하기 위해서는 .detach()를 사용한다. 코드블럭을 with torch.no_grad():로 감쌀 수 있는데, gradient는 필요 없지만, 학습 가능한 매개변수를 갖는 모델을 평가할 때 유용하다.

* Function

tensor와 Function은 서로 연결되어 있는데 모든 연산과정을 부호화(encode)하여 acycle graph를 생성한다.

tensor의 .graph_fn 속성은 tensor를 생성한 Function속성을 참조한다. 그러나 사용자가 만든 tensor의 .graph_fn 값은 None이다.

도함수를 계산하기 위해서는 .backward()를 호출하고 tensor가 여러 요소를 갖고 있을 경우에는 tensor의 모양을 gradient의 인자로 지정할 필요가 존재한다

In [194]:
import torch

In [195]:
# x에서 이뤄진 연산을 track하도록 True로 설정
x = torch.ones(2, 2, requires_grad = True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [196]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [197]:
# y는 사용자 지정함수가 아니므로 grad_fn을 갖고, x는 사용자 지정함수이므로 None값을 갖는다.
print(y.grad_fn, x.grad_fn)

<AddBackward0 object at 0x7f719dbcf780> None


In [198]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


.requires_grad _ 는 기존의 tensor의 requies_grad 값을 in-place하여 변경함

입력값이 지정되지 않을 경우 기본 값은 False

In [199]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


## Gradient
역전파(backprop)에서 out은 하나의 스칼라 값(mean을 취했으므로)을 가지고 있기 때문에 out.backward()는 out.backward(torch.tensor(1.))과 동일하다

여기서 x는 앞서 $y = x + 2$ $z = y \times y \times 3$ out = z.mean()을 통해  변환을 거쳤으므로, $d(out) \over dx$를 출력한 x.grad는 4.5에 해당한다.

In [200]:
out.backward()

In [201]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [205]:
x = torch.randn(3, requires_grad = True)

y = x * 2
while y.data.norm() < 1000:
  y = y * 2
print(y)

tensor([ -210.7840, -1391.3506,  1079.1760], grad_fn=<MulBackward0>)


y가 스칼라 값이 아니기 때문에 torch.autograd는 전체 야코비안을 계산할 수 없다. 그러나, vector-jacobian곱은 backward의 인자로 받아 계산이 가능하다

In [206]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype = torch.float)
y.backward(v)
print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


코드블럭을 감싸서 기록의 추적을 막을 수 있음

In [208]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
  print((x**2).requires_grad)

True
True
False


.detach()를 호출하여 내용물은 같지만 require_grad가 다른 새로운 tensor를 가져올 수 있음

In [209]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)
